Data Preperation

In [31]:
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.decomposition import PCA

In [8]:
training = pd.read_csv('HMAHCC_COMP.csv')

C:\Users\cgn31\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
class DataPrep:
    def __init__(self, df):
        self.raw_df = df
    
    def seperate_frame(self):
        new_diagnosis_col = ['New diagnosis - CPD',
                         'New diagnosis - Hypertension', 
                         'New diagnosis - Top 5', 
                         'New diagnosis - CAD', 
                         'New diagnosis - CHF', 
                         'New diagnosis - Diabetes']

        call_col = ['Inbound Call by Prov',
                    'Inbound Call by Mbr',
                    'Inbound Call by Other']

        non_rx_claim_col = ['Surgery',
                            'Fully Paid Claim']

        rx_claim_col = ['RX Claim - New Drug',
                        'RX Claim - First Time Mail Order']
        
        self.new_diagnosis = self.raw_df[self.raw_df['event_descr'].isin(new_diagnosis_col)]

        self.new_diagnosis = self.new_diagnosis.drop(['event_attr6',
                                                      'event_attr7',
                                                      'event_attr8',
                                                      'event_attr9',
                                                      'event_attr10',
                                                      'PAY_DAY_SUPPLY_CNT',
                                                      'PAYABLE_QTY',
                                                      'MME',
                                                      'DRUG_TYPE',
                                                      'Specialty',
                                                      'Specialty2',
                                                      'Specialty3'],
                                                       axis = 1)

        self.new_diagnosis.columns = ['id',
                                 'event_descr',
                                 'diagnosis',
                                 'place_of_treatment',
                                 'charge_amount',
                                 'net_paid_amount',
                                 'member_responsible_amount',
                                 'Days']
        
        self.call = self.raw_df[self.raw_df['event_descr'].isin(call_col)]

        self.call = self.call.drop(['event_attr6',
                          'event_attr7',
                          'event_attr8',
                          'event_attr9',
                          'event_attr10',
                          'PAY_DAY_SUPPLY_CNT',
                          'PAYABLE_QTY',
                          'MME',
                          'DRUG_TYPE',
                          'Specialty',
                          'Specialty2',
                          'Specialty3'],
                           axis = 1)

        self.call.columns = ['id',
                        'event_descr',
                        'call_category',
                        'inquiry_reason_description',
                        'disposition_description',
                        'origin',
                        'location',
                        'Days']
        
        self.non_rx_claim = self.raw_df[self.raw_df['event_descr'].isin(non_rx_claim_col)]

        self.non_rx_claim = self.non_rx_claim.drop(['event_attr6',
                                          'event_attr7',
                                          'event_attr8',
                                          'event_attr9',
                                          'event_attr10',
                                          'PAY_DAY_SUPPLY_CNT',
                                          'PAYABLE_QTY',
                                          'MME',
                                          'DRUG_TYPE',
                                          'Specialty',
                                          'Specialty2',
                                          'Specialty3'],
                                           axis = 1)

        self.non_rx_claim.columns = ['id',
                                'event_descr',
                                'diagnosis',
                                'place_of_treatment',
                                'charge_amount',
                                'net_paid_amount',
                                'member_responsible_amount',
                                'Days']
        
        self.new_provider = self.raw_df[self.raw_df['event_descr']=='New provider']
        self.new_provider = self.new_provider.iloc[:,:2]
        
        self.rx_claim = self.raw_df[self.raw_df['event_descr'].isin(rx_claim_col)]

        self.rx_claim = self.rx_claim.drop(['event_attr6',
                          'event_attr7'],
                           axis = 1)

        self.rx_claim.columns = ['id',
                            'event_descr',
                            'gpi_drug_group6_id',
                            'gpi_drug_class_description',
                            'brand_name',
                            'drug_group_id',
                            'generic_name',
                            'drug_group_description',
                            'member_responsible_amount',
                            'gpi_drug_group8_id',
                            'Days',
                            'PAY_DAY_SUPPLY_CNT',
                            'PAYABLE_QTY',
                            'MME',
                            'DRUG_TYPE',
                            'Specialty',
                            'Specialty2',
                            'Specialty3']
        
        self.rx_paid = self.raw_df[self.raw_df['event_descr']=='RX Claim - Paid']

        self.rx_paid = self.rx_paid.drop(['event_attr2',
                                'event_attr7'],
                                 axis = 1)

        self.rx_paid.columns = ['id',
                           'event_descr',
                           'gpi_drug_class_description',
                           'rx_cost',
                           'net_paid_amount',
                           'brand_name',
                           'drug_group_description',
                           'generic_name',
                           'member_responsible_amount',
                           'gpi_drug_group8_id',
                           'Days',
                           'PAY_DAY_SUPPLY_CNT',
                           'PAYABLE_QTY',
                           'MME',
                           'DRUG_TYPE',
                           'Specialty',
                           'Specialty2',
                           'Specialty3']
        
        self.rx_reject = self.raw_df[self.raw_df['event_descr']=='RX Claim - Rejected']

        self.rx_reject = self.rx_reject.drop(['PAY_DAY_SUPPLY_CNT',
                                    'PAYABLE_QTY',
                                    'MME',
                                    'DRUG_TYPE',
                                    'Specialty',
                                    'Specialty2',
                                    'Specialty3'],
                                     axis =1)

        self.rx_reject.columns = ['id',
                             'event_descr',
                             'status_code',
                             'diagnosis',
                             'cob',
                             'claim_tier',
                             'brand_name',
                             'generic_name',
                             'ndc_id',
                             'pay_day_supply_count',
                             'member_responsible_amount',
                             'gpi_drug_group8_id',
                             'Days']
        
    def get_opioid_data(self):
        self.opioid_data = self.raw_df[self.raw_df['PAY_DAY_SUPPLY_CNT'].notnull()]
        self.opioid_data_grouped = self.opioid_data.groupby(by='id')
        
    def LTOT(self):
        
        def get_LTOT(ID):
            try:
                group = self.opioid_data_grouped.get_group(ID)
                frame = group[['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
                frame = frame[frame['Days']>=0]
                frame['drugs until'] = (frame['Days'] + frame['PAY_DAY_SUPPLY_CNT']).astype(int)
                frame['range'] = frame.apply(lambda x : range(x['Days'].astype(int),x['drugs until'].astype(int)),1)

                concat = concatenated = chain(*list(frame['range']))
                concat = set(concat)

                day_frame = pd.DataFrame(columns = ['Days', 'Has Drug?'])
                day_frame['Days'] = range(max(concat)+1)
                day_frame['Has Drug?'] = (day_frame['Days'].isin(concat))

                for n in range(180,len(day_frame)+1):
                    frame_slice = day_frame.iloc[n-180:n]
                    drug_days = np.sum(frame_slice['Has Drug?'])

                    if drug_days >= 162:
                        return (True, n-180, n)

                return (False, np.nan, np.nan)
            except:
                return (np.nan, np.nan, np.nan)
        
        id_list = self.raw_df['id'].drop_duplicates().values
        
        response_variable = pd.DataFrame(id_list, columns = ['id'])
        response_variable['LTOT'] = response_variable['id'].map(get_LTOT)
        
        response_variable[['LTOT', 'Begining Date', 'End Date']] = \
        pd.DataFrame(response_variable['LTOT'].tolist(), index = response_variable.index)
        
        self.response_variable = response_variable
    
    
    def main_feature_extraction(self): 
        opioid_all_time = self.rx_paid[self.rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['generic_name'].value_counts()
        mask = self.rx_paid['generic_name'].map(lambda x: x in opioid_all_time.index.values)
        true_opioid = self.rx_paid[mask]
        
        opioid_grouped = true_opioid.groupby(by=['id'])

        idtestlist = true_opioid['id'].drop_duplicates()
        features3 = pd.DataFrame()

        for ID in idtestlist:
            tmp = opioid_grouped.get_group(ID)

            # MME (per day) on day 0
            # Suuply_CNT on day 0
            on_day0 = tmp[tmp['Days'] == 0] 
            if not on_day0.empty:
                MME0 = on_day0['MME'].values[0]
                SC0 = on_day0['PAY_DAY_SUPPLY_CNT'].values[0]
                PQ0 = on_day0['PAYABLE_QTY'].values[0]
            else:
                MME0 = 0
                SC0 = 0
                PQ0 = 0

            # max MME (per day) prior to day 0
            # average MME (per day) prior to day 0
            # Total Supply_CNT prior to day 0
            prior_day0 = tmp[tmp['Days'] < 0]
            if not prior_day0.empty:
                maxMME = np.nanmax(prior_day0['MME'].values)
                avgMME = np.nanmean(prior_day0['MME'].values)
                totalSC = np.nansum(prior_day0['PAY_DAY_SUPPLY_CNT'].values)
                totalPQ = np.nansum(prior_day0['PAYABLE_QTY'].values)
            else:
                maxMME = 0
                avgMME = 0
                totalSC = 0
                totalPQ = 0

            output = pd.DataFrame({'MME_on_day0': MME0, 
                                 'SUPPLY_CNT_on_day0': SC0,
                                   'PAYABLE_QTY_on_day0': PQ0,
                                 'max_MME_prior': maxMME,
                                 'avg_MME_prior': avgMME,
                                 'total_SUPPLY_CNT_prior': totalSC,
                                  'total_PAYABLE_QTY_prior': totalPQ},
                                  index = [ID])

            features3 = features3.append(output, sort=False)

        # MME_on_day0, max_MME_prior, avg_MME_prior has some missing value, fill with medians
        features3['MME_on_day0'] = features3['MME_on_day0'].fillna(np.nanmedian(features3['MME_on_day0']))
        features3['max_MME_prior'] = features3['max_MME_prior'].fillna(np.nanmedian(features3['max_MME_prior']))
        features3['avg_MME_prior'] = features3['avg_MME_prior'].fillna(np.nanmedian(features3['avg_MME_prior']))

        # add one more feature: supply_times
        supply_times = true_opioid[true_opioid['Days']<=0].groupby(by=['id'])['PAY_DAY_SUPPLY_CNT'].count()
        supply_times = pd.DataFrame(supply_times)
        supply_times.columns = ['supply_times']
        features3 = features3.merge(supply_times, left_on=features3.index.values, right_on=supply_times.index.values)
        features3 = features3.set_index('key_0')
        
        self.main_features = features3
        
    def generic_feature_extraction(self):
        opioid_all_time = self.rx_paid[self.rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['generic_name'].value_counts()
        mask = self.rx_paid['generic_name'].map(lambda x: x in opioid_all_time.index.values)
        true_opioid = self.rx_paid[mask] 
        
        opioid2_grouped = true_opioid[true_opioid['Days'] == 0].groupby(by=['id'])

        idtestlist = true_opioid[true_opioid['Days'] == 0]['id'].drop_duplicates()

        ## those commented-out codes are used to get other entry values
        # def product_sum(df):
        #     return(df['MME'].values.dot(df['PAY_DAY_SUPPLY_CNT']e.values))

        features = pd.DataFrame()
        for ID in idtestlist:
            tmp = opioid2_grouped.get_group(ID)
            output = pd.DataFrame(tmp.groupby(by='generic_name')['PAY_DAY_SUPPLY_CNT'].agg(np.nansum)).T
        #     output = output.iloc[0:1,:]
            output.index = [ID]
            # features = pd.concat([output, features], axis=1, sort=False)
            features = features.append(output, sort=False)

        features = features.fillna(0)
        
        pca = PCA(n_components = 10) 
        X10D = pca.fit_transform(features)
        
        features_matthew_generic = pd.DataFrame(X10D, index=features.index.values)
        features_matthew_generic.columns = ['generic_pc{}'.format(x) for x in features_matthew_generic.columns]
        
        self.generic_features = features_matthew_generic

In [33]:
%%time
training_class = DataPrep(training)
training_class.seperate_frame()
training_class.get_opioid_data()
training_class.main_feature_extraction()
training_class.generic_feature_extraction()

C:\Users\cgn31\Anaconda3\lib\site-packages\ipykernel_launcher.py:249: RuntimeWarning: All-NaN slice encountered
C:\Users\cgn31\Anaconda3\lib\site-packages\ipykernel_launcher.py:250: RuntimeWarning: Mean of empty slice


Wall time: 6min 13s


In [ ]:
#training_class.LTOT()

In [ ]:
#training_class.response_variable

In [34]:
training_class.main_features

,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,supply_times
key_0,,,,,,,,
ID10010854159,15.000000,5.0,15.0,0.000000,0.000000,0.0,0.0,1
ID10013863216,10.000000,90.0,180.0,0.000000,0.000000,0.0,0.0,1
ID10024447278,50.000000,3.0,20.0,22.500000,20.000000,15.0,40.0,4
ID1002482139,60.000000,30.0,120.0,90.000000,37.254464,789.0,2895.0,29
ID1003386406,20.000000,15.0,60.0,50.000000,50.000000,3.0,20.0,2
ID10036289882,37.500000,2.0,15.0,0.000000,0.000000,0.0,0.0,1
ID10036954381,37.500000,2.0,15.0,37.500000,37.500000,2.0,15.0,2
ID10055739763,15.000000,5.0,15.0,30.000000,30.000000,3.0,12.0,2
ID10074598346,20.000000,5.0,20.0,40.000000,25.842105,452.0,1480.0,20


In [35]:
training_class.generic_features

,generic_pc0,generic_pc1,generic_pc2,generic_pc3,generic_pc4,generic_pc5,generic_pc6,generic_pc7,generic_pc8,generic_pc9
ID10010854159,-5.827083,-1.440194,-2.612607,-1.891909,-0.929671,-0.359510,-0.474685,-1.747752,-0.471185,-0.390152
ID10013863216,78.911681,0.921673,1.581688,1.100533,0.430629,0.207605,0.223252,0.929939,0.234209,0.204999
ID10024447278,-10.858223,-1.648665,-3.029095,-2.332854,-1.340008,-0.729227,-1.924404,0.671163,-0.374995,-0.352461
ID1002482139,-10.945533,-1.754316,-3.247826,-2.562327,-1.426616,-0.615339,-0.887676,-3.176651,-2.343673,29.255099
ID1003386406,4.142184,-1.162327,-2.119161,-1.539857,-0.769636,-0.292790,-0.392575,-1.432729,-0.388198,-0.320134
ID10036289882,-10.921630,0.197789,-2.020023,-1.790856,-0.906053,-0.355247,-0.467977,-1.738096,-0.472536,-0.392934
ID10036954381,-10.921630,0.197789,-2.020023,-1.790856,-0.906053,-0.355247,-0.467977,-1.738096,-0.472536,-0.392934
ID10055739763,-11.086501,2.863164,-0.761061,-1.375237,-0.750601,-0.298814,-0.396333,-1.487345,-0.412321,-0.344594
ID10074598346,-11.086501,2.863164,-0.761061,-1.375237,-0.750601,-0.298814,-0.396333,-1.487345,-0.412321,-0.344594
ID10081072715,-11.086501,2.863164,-0.761061,-1.375237,-0.750601,-0.298814,-0.396333,-1.487345,-0.412321,-0.344594


In [ ]:
training_class.opioid_data

From here we work with the data: